# DAPA Tutorial #4: Value - Sentinel-2

## Load environment variables
Please make sure that the environment variable "DAPA_URL" is set in the `custom.env` file. You can check this by executing the following block. 

If DAPA_URL is not set, please create a text file named `custom.env` in your home directory with the following input: 
>DAPA_URL=YOUR-PERSONAL-DAPA-APP-URL

In [1]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


## Check notebook compabtibility
**Please note:** If you conduct this notebook again at a later time, the base image of this Jupyter Hub service can include newer versions of the libraries installed. Thus, the notebook execution can fail. This compatibility check is only necessary when something is broken. 

In [2]:
from edc import check_compatibility
check_compatibility("user-0.19.6")

This notebook is compatible with this base image version (user-0.19.6).

## Load libraries
Python libraries used in this tutorial will be loaded.

In [3]:
import os
import xarray as xr
import pandas as pd
import requests
import matplotlib
from ipyleaflet import Map, Rectangle, Marker, DrawControl, basemaps, basemap_to_tiles
%matplotlib inline

## Set DAPA endpoint
Execute the following code to check if the DAPA_URL is available in the environment variable and to set the `/dapa` endpoint. 

In [4]:
service_url = None
dapa_url = None

if 'DAPA_URL' not in os.environ:
    print('!! DAPA_URL does not exist as environment variable. Please make sure this is the case - see first block of this notebook! !!')
else:    
    service_url = os.environ['DAPA_URL']
    dapa_url = '{}/{}'.format(service_url, 'oapi')
    print('DAPA path: {}'.format(dapa_url.replace(service_url, '')))

DAPA path: /oapi


## Get collections supported by this endpoint
This request provides a list of collections. The path of each collection is used as starting path of this service.

In [5]:
collections_url = '{}/{}'.format(dapa_url, 'collections')
collections = requests.get(collections_url, headers={'Accept': 'application/json'})

print('DAPA path: {}'.format(collections.url.replace(service_url, '')))
collections.json()

DAPA path: /oapi/collections


['/oapi/collections/DEM/',
 '/oapi/collections/L8L1C/',
 '/oapi/collections/S1GRD/',
 '/oapi/collections/S2L1C/',
 '/oapi/collections/S2L2A/',
 '/oapi/collections/S3OLCI/',
 '/oapi/collections/S3SLSTR/',
 '/oapi/collections/S5PL2/',
 '/oapi/collections/MODIS/']

## Get fields of collection Sentinel-2 L2A
The fields (or variables in other DAPA endpoints - these are the bands of the raster data) can be retrieved in all requests to the DAPA endpoint. In addition to the fixed set of fields, "virtual" fields can be used to conduct math operations (e.g., the calculation of indices). 

In [6]:
collection = 'S2L2A'

fields_url = '{}/{}/{}/{}'.format(dapa_url, 'collections', collection, 'dapa/fields')
fields = requests.get(fields_url, headers={'Accept': 'application/json'})

print('DAPA path: {}'.format(fields.url.replace(service_url, '')))
fields.json()

DAPA path: /oapi/collections/S2L2A/dapa/fields


[{'id': 'B01', 'wavelength': 442.7},
 {'id': 'B02', 'wavelength': 492.4},
 {'id': 'B03', 'wavelength': 559.8},
 {'id': 'B04', 'wavelength': 664.6},
 {'id': 'B05', 'wavelength': 704.1},
 {'id': 'B06', 'wavelength': 740.5},
 {'id': 'B07', 'wavelength': 782.8},
 {'id': 'B08', 'wavelength': 832.8},
 {'id': 'B8A', 'wavelength': 864.7},
 {'id': 'B09', 'wavelength': 945.1},
 {'id': 'B11', 'wavelength': 1613.7},
 {'id': 'B12', 'wavelength': 2202.4},
 {'id': 'SCL', 'wavelength': None},
 {'id': 'SNW', 'wavelength': None},
 {'id': 'CLD', 'wavelength': None}]

## Retrieve an NDVI value extraced for a single point

### Set DAPA URL and parameters
The output of this request is a single value requested from a point of interest (`value/position` endpoint). As the input collection (S2L2A) is a multi-temporal raster and the requested geometry is a point, temporal aggregation is conducted.

To retrieve a single value of a point, the parameter `point` needs to be provided. For the temporal aggregation, the parameter `aggregate` needs to include the aggregation functions splitted by comma. The `time` parameter allows to aggregate data only within a specific time span. The band (`field`) from which the point is being extracted needs to be specified as well.

In [7]:
# DAPA URL
url = '{}/{}/{}/{}'.format(dapa_url, 'collections', collection, 'dapa/value/position')

# Parameters for this request
params = {
    'point': '11.49,48.05',
    'time': '2018-04-01T00:00:00Z/2018-07-01T00:00:00Z',
    'fields': 'NDVI=(B04-B08)/(B04%2BB08)',
    'aggregate': 'min,max,avg'
}

# show point in the map
location = list(reversed([float(coord) for coord in params['point'].split(',')]))

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=location,
    zoom=10
)

marker = Marker(location=location, draggable=False)
m.add_layer(marker)

m

Map(center=[48.05, 11.49], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Build request URL and conduct request

In [8]:
params_str = "&".join("%s=%s" % (k, v) for k,v in params.items())
r = requests.get(url, params=params_str)

print('DAPA path: {}'.format(r.url.replace(service_url, '')))
print('Status code: {}'.format(r.status_code))

DAPA path: /oapi/collections/S2L2A/dapa/value/position?point=11.49,48.05&time=2018-04-01T00:00:00Z/2018-07-01T00:00:00Z&fields=NDVI=(B04-B08)/(B04%2BB08)&aggregate=min,max,avg
Status code: 200


### Show content (one value for each aggregation method)
The response of this request returns plain text with values splitted by comma. The order of the values relates to the order of the fields specified in the `fields` parameter in combination with the order of the `aggregation` functions. Additional output formats (e.g., CSV with headers included) will be integrated within the testbed activtiy. 

In [9]:
values = r.text
values

'-0.8713367,0.039432675,-0.4004872'

In [10]:
print('%s: %s' % (params['aggregate'].split(',')[0], values.split(',')[0]))
print('%s: %s' % (params['aggregate'].split(',')[1], values.split(',')[1]))
print('%s: %s' % (params['aggregate'].split(',')[2], values.split(',')[2]))

min: -0.8713367
max: 0.039432675
avg: -0.4004872
